In [1]:
from pyspark.sql import SparkSession

In [27]:
from pyspark.context import SparkContext

In [28]:
sc = SparkContext.getOrCreate()

In [31]:
from pyspark.sql import functions as Fn

In [2]:
spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [5]:
df = spark.read.parquet("C:\\Prafull\\Python Training\\Ipfcode\\source\\freud\\NFS_DBO\\Arrears_Contact_Detail_Table\\*")

In [6]:
df.count()

743059

In [8]:
column_names = df.schema.names

In [9]:
table_schema = df.schema

In [10]:
df_mapping = spark.read.option("multiline", "true").json("C:\\Prafull\\Python Training\\Ipfcode\\mappings\\romania\\freud\\nfs_dbo\\arrears_contact_detail_table_mapping.json")

In [11]:
id_col = df_mapping.select("id_column").collect()[0][0]

In [15]:
pii_columns = df_mapping.select("pii_columns").collect()[0][0]

In [16]:
mapping = df_mapping.select("gdpr_mappings").collect()[0][0]

In [20]:
import ast

In [23]:
 id_q = ast.literal_eval("[[2252, 1704], 1704000032986]")

In [24]:
print(id_q)

[[2252, 1704], 1704000032986]


In [25]:
if type(id_q) is not tuple:
                id_q = tuple([id_q])

In [29]:
df_filter = sc.parallelize(id_q).toDF()

In [32]:
for x in range(len(id_col)):
                df_filter = df_filter.withColumn(id_col[x], Fn.col('_1')[x])

In [33]:
print(df_filter)

DataFrame[_1: array<bigint>, _2: bigint, arrears_contact_detail_id: bigint, arrears_contact_detail_db_id: bigint]


In [34]:
df_add = df.join(Fn.broadcast(df_filter), id_col, "inner")

In [35]:
print(df_add)

DataFrame[arrears_contact_detail_id: bigint, arrears_contact_detail_db_id: bigint, agreed_rate: double, agreed_rate_start_date: string, agreed_rate_end_date: string, army_personnel_details: string, asset_information: string, debt_agency_company_code: string, customer_contact_db_id: bigint, customer_contact_id: bigint, customer_id: bigint, debt_agency_code: string, deleted_flag: bigint, person_contacted_code: string, repayment_frequency_code: string, dt_generation_id: bigint, dt_transaction_id: string, dt_write_bln: boolean, last_updated_datetime: string, last_updated_id: bigint, last_updated_timestamp: string, additional_information: string, filepath: string, row_number: bigint, _1: array<bigint>, _2: bigint]


In [37]:
df.show(5)

+-------------------------+----------------------------+-----------+----------------------+--------------------+----------------------+-----------------+------------------------+----------------------+-------------------+-------------+----------------+------------+---------------------+------------------------+----------------+--------------------+------------+---------------------+---------------+----------------------+----------------------+--------------------+----------+
|arrears_contact_detail_id|arrears_contact_detail_db_id|agreed_rate|agreed_rate_start_date|agreed_rate_end_date|army_personnel_details|asset_information|debt_agency_company_code|customer_contact_db_id|customer_contact_id|  customer_id|debt_agency_code|deleted_flag|person_contacted_code|repayment_frequency_code|dt_generation_id|   dt_transaction_id|dt_write_bln|last_updated_datetime|last_updated_id|last_updated_timestamp|additional_information|            filepath|row_number|
+-------------------------+-------------